In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import itertools as it
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from sklearn.model_selection import cross_val_score



# df.shape
# df.head()

In [ ]:
df = pd.read_csv("train.csv")
dfTest = pd.read_csv("test.csv")
target = df['Survived']
passId = dfTest['PassengerId']
dfTest.drop(['PassengerId', 'Ticket'], axis =1, inplace = True)




quasiConst = [col for col in df.columns if  ((df[col].dtype == 'int64' or df[col].dtype == 'float64') and df[col].std() == 0) or (df[col].dtype == 'object' and len(df[col].unique()) == 1 )]
print(len(quasiConst))
quasiConst

In [ ]:
df.head(10)
dfTest.head(30)


In [ ]:
df.isnull().sum()
dfTest.isnull().sum()

dfTest[dfTest['Age'].isnull()]

In [ ]:
df['Cabin'].fillna('Missing', inplace=True)
print df.isnull().sum()
df['Cabin'] = df['Cabin'].astype(str).str[0]
dictMon = df.groupby(['Cabin'])['Survived'].mean().to_dict()
df['Cabin'] = df['Cabin'].map(dictMon)

dfTest['Cabin'].fillna('Missing', inplace=True)
dfTest['Cabin'] = dfTest['Cabin'].astype(str).str[0]
dfTest['Cabin'] = dfTest['Cabin'].map(dictMon)

In [ ]:
df.loc[df['Embarked'].isnull()]
df['Embarked'].value_counts().sort_values(ascending=False)
domVal = df['Embarked'].value_counts().sort_values(ascending=False).index[0]
df['Embarked'].fillna(value=domVal, inplace = True)
df['Embarked'].value_counts().sort_values(ascending=False)
df['Embarked'].value_counts(dropna = False).sort_values(ascending=False).plot(kind='bar')

In [ ]:
dEmb = df.groupby(['Embarked'])['Survived'].mean().to_dict()
df['Embarked'] = df['Embarked'].map(dEmb)
dfTest['Embarked'] = dfTest['Embarked'].map(dEmb)

In [ ]:
len(df['Ticket'].unique())
df.drop(['Ticket'], inplace = True, axis = 1)
df.head()

In [ ]:
df['Fare'].value_counts().sort_index()
plt.show()
lstBins = [-1, 20, 40, 60, 80, 100, 150, 1000]
df['Fare'] = pd.cut(df['Fare'], bins = lstBins, labels = range(1, len(lstBins)))

dfTest['Fare'].value_counts().sort_index()
dfTest['Fare'] = dfTest['Fare'].fillna(df['Fare'].value_counts().sort_values(ascending=False).index[0])
dfTest['Fare'] = pd.cut(dfTest['Fare'], bins = lstBins, labels = range(1, len(lstBins)))


In [ ]:
df['Name'] = df['Name'].str.split('[,.]').str[1]
names = list(df['Name'].unique())
print 'train, ' , names

dfTest['Name'] = dfTest['Name'].str.split('[,.]').str[1]
namesTest = list(dfTest['Name'].unique())
print 'test, ' , namesTest

In [ ]:
# for col in ['SibSp' , 'Parch']:
#     plt.show()
#     df.groupby([col])['Survived'].mean().plot(kind='bar')

In [ ]:
# dAge = df.groupby('Name')['Age'].median()
# df['Age_'] = df['Age'].fillna(df['Age'].map(dAge))
# dfTest['Age_'] = dfTest['Age'].fillna(dfTest['Age'].map(dAge))
dfTest['Age'].isnull().sum()
df['Age'] = df.groupby(['Name'])['Age'].apply(lambda x: x.fillna(x.median()))
dfTest['Age'] = dfTest.groupby(['Name'])['Age'].apply(lambda x: x.fillna(x.median()))

val = dfTest['Age'].value_counts().sort_values(ascending=False).index[0]
dfTest['Age'] = dfTest['Age'].fillna(val)

dfTest['Age'].isnull().sum()

df.head(20)
dfTest.head(20)

In [ ]:
df.groupby('Name')['Age'].mean().plot(kind='bar')
plt.show()
#dfTest.groupby('Name')['Age'].mean().plot(kind='bar')

In [ ]:
dNameSurv = df.groupby(['Name'])['Survived'].mean().to_dict()
dNameSurv
df['Name'] = df['Name'].map(dNameSurv)
dfTest['Name'] = dfTest['Name'].map(dNameSurv)
df['Name'].median()

dfTest['Name'] = dfTest['Name'].fillna(df['Name'].value_counts().sort_values(ascending=False).index[0])

In [ ]:
# dNamesMap = {}
# diffNames = list(set(names + namesTest))
# for n in diffNames:
#     if n in [' Miss', ' Mrs', ' Mr', ' Ms']:
#         dNamesMap[n] = 1
#     elif n in [' Mme', ' Mlle', ' Major', ' Col', ' Dr', ' Rev']:
#         dNamesMap[n] = 2
#     else:
#         dNamesMap[n] = 3
        
# df['Name'] = df['Name'].map(dNamesMap)
# dfTest['Name'] = dfTest['Name'].map(dNamesMap)

In [ ]:
#df['Age'] = df.groupby(['Name'])['Age'].apply(lambda x: x.fillna(x.median()))

In [ ]:
df['Female'] = np.where(df['Sex'] == 'female', 1, 0)
df.drop(['Sex'], inplace = True, axis = 1)

dfTest['Female'] = np.where(dfTest['Sex'] == 'female', 1, 0)
dfTest.drop(['Sex'], inplace = True, axis = 1)

In [ ]:
quasConstVars = []
for col in df.columns:
    domVal = (df[col].value_counts() / float(len(df))).sort_values(ascending=False).values[0]
    print col, domVal
    if domVal > .9:
        quasConstVars.append(col)
quasConstVars

In [ ]:
for col in df.columns:
    print col
    #plt.show()
    #df[col].value_counts().sort_index(ascending=True).plot(kind='bar')
    df[col].value_counts().sort_index(ascending=True)

In [ ]:
dfTest['Age'].value_counts().sort_index()

In [ ]:
dfTest.head(10)
binAge = [-1] + [3*n for n in range(1,30)]

len(range(1,18))
len(binAge)
df['Age'] = pd.cut(df.Age, bins = binAge, labels = range(1,len(binAge)))
dfTest['Age'] = pd.cut(dfTest.Age, bins = binAge, labels = range(1,len(binAge)))
dfTest.head(10)

In [ ]:
df.corr().abs().unstack()[df.corr().abs().unstack() > .5]

In [ ]:
df['Pclass'].unique()
dfTest['Pclass'].unique()
df.columns

In [ ]:
df.drop(['Fare'], inplace = True, axis = 1)
dfTest.drop(['Fare'], inplace = True, axis = 1)

dClass = df.groupby(['Pclass'])['Survived'].mean().to_dict()



df['Pclass'] = df['Pclass'].map(dClass)
dfTest['Pclass'] = dfTest['Pclass'].map(dClass)



df.drop(['Survived', 'PassengerId'] , axis = 1, inplace = True)

df.isnull().sum()
dfTest.isnull().sum()

In [ ]:
df['Family'] = np.where(np.logical_or(df['SibSp'] > 0, df['Parch'] > 0 ), 1, 0)
dfTest['Family'] = np.where(np.logical_or(dfTest['SibSp'] > 0, dfTest['Parch'] > 0 ), 1, 0)

df.drop(['SibSp', 'Parch'] , axis = 1, inplace = True)
dfTest.drop(['SibSp', 'Parch'] , axis = 1, inplace = True)

dfTest.head(10)
df.head(10)

In [ ]:
# nEst = range(1, 50)
# mD = range(1, 50)
# min_split = (2, 50)
# param_grid = dict(max_depth = mD, min_samples_split = min_split, n_estimators = nEst)
# grid = GridSearchCV(RandomForestClassifier(random_state=0), param_grid, cv = 10, scoring = 'accuracy', verbose = 4)

# grid.fit(df, target)
# print grid.best_score_
# print grid.best_params_


In [ ]:
# sol = ['adam', 'lbfgs']
# act = ['identity', 'logistic', 'tanh', 'relu']
# maxit = [200, 400]
# hid = range(1, 6) + [x for x in it.product((1, 2, 3, 4),repeat=2)]
# param_grid = dict(solver = sol, activation = act, max_iter = maxit, hidden_layer_sizes = hid)
# grid = GridSearchCV(MLPClassifier(random_state=0), param_grid, cv = 10, scoring = 'accuracy', verbose = 4)

# grid.fit(df, target)
# print grid.best_score_
# print grid.best_params_

In [ ]:
# sol = ['lbfgs']
# act = ['logistic']
# maxit = [200, 400]
# hid = range(6, 10) + [x for x in it.product((5, 6, 7),repeat=2)]
# param_grid = dict(solver = sol, activation = act, max_iter = maxit, hidden_layer_sizes = hid)
# grid = GridSearchCV(MLPClassifier(random_state=0), param_grid, cv = 10, scoring = 'accuracy', verbose = 4)
# grid.fit(df, target)
# print grid.best_score_
# print grid.best_params_

In [ ]:
c = [.8, 1, 1.2]
ker = ['rbf','linear' , 'poly', 'sigmoid']
gam = ['auto', .3, .5, .8]
deg = [1, 2]
coef = [0.0, 0.2, 0.5]
boolean = [True, False]

param_grid = dict(C = c, kernel = ker, gamma = gam, degree = deg, coef0 = coef, shrinking = boolean, probability = boolean)
grid = GridSearchCV(SVC(random_state=0), param_grid, cv = 10, scoring = 'accuracy', verbose = 6)

grid.fit(df, target)
print grid.best_score_
print grid.best_params_

In [ ]:
# .7945
# clf = SVC(kernel= 'poly', C = 0.8, probability =  True, degree = 2, shrinking = True, coef0 = 0.2, gamma = 0.5, random_state=0)#.fit(dfMergedTrain, target) 
# scores = cross_val_score(clf, df, target, cv=10)
# print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)) #Accuracy: 0.70 (+/- 0.00)

In [ ]:
# clf = SVC(kernel= 'poly', C = 0.8, probability =  True, degree = 2, shrinking = True, coef0 = 0.2, gamma = 0.5, random_state=0)#.fit(dfMergedTrain, target) 
# scores = cross_val_score(clf, df, target, cv=10)
# print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)) #Accuracy: 0.70 (+/- 0.00)

In [ ]:
# clf = SVC(kernel= 'poly', C = 0.8, probability =  True, degree = 2, shrinking = True, coef0 = 0.2, gamma = 0.5, random_state=0).fit(df, target) # 65%
# # clf = MLPClassifier(activation='identity', max_iter=700, solver='adam', hidden_layer_sizes=(2, 4), random_state=0).fit(dfMergedTrain, target)
# predArray = clf.predict(dfTest)
# #updatedTestArray = array(dfTest.iloc[:, :])
# with open("output.csv","w") as f:
#     f.write("PassengerId,Survived\n")
#     for x in range(0, len(predArray)):
#         predVal = predArray[x]
#         # f.write(str(dfMergedTest.iloc[x,1]) + "," + str(predVal)[1:-1] +"\n")
#         f.write(str(passId[x]) + "," + str(predVal) +"\n")
#         # f.write(str(buildingId[x]) + "," + str(1) +"\n")